### Download required dependencies using this code

In [1]:
# !pip install syrupy pytest-regressions ipytest

In [1]:
import pytest

In [2]:
import ipytest
ipytest.autoconfig()

### The current directory should be empty.

### The following test should fail- saying that no snapshots are present.

In [6]:
%%ipytest -vv 
pytest_plugins = "syrupy"

from typing import Any, List, Tuple
import pytest
import numpy as np
from syrupy.data import SnapshotCollection
from syrupy.extensions.single_file import SingleFileSnapshotExtension
import pandas as pd
from syrupy.location import PyTestLocation
from syrupy.types import SerializableData, SerializedData, SnapshotIndex
from gettext import gettext


class NumpySnapshotExtenstion(SingleFileSnapshotExtension):
    _file_extension = "dat"

    def matches(self, *, serialized_data, snapshot_data):
        try:
            if np.testing.assert_allclose(
                np.array(snapshot_data), np.array(serialized_data)
            )  is not None:
                return False
            else: return True
            
        except:
            return False

    def _read_snapshot_data_from_location(
        self, *, snapshot_location: str, snapshot_name: str, session_id: str
    ):
        # see https://github.com/tophat/syrupy/blob/f4bc8453466af2cfa75cdda1d50d67bc8c4396c3/src/syrupy/extensions/base.py#L139
        try:
            return np.loadtxt(snapshot_location).tolist()
        except OSError:
            return None

    @classmethod
    def _write_snapshot_collection(
        cls, *, snapshot_collection: SnapshotCollection
    ) -> None:
        # see https://github.com/tophat/syrupy/blob/f4bc8453466af2cfa75cdda1d50d67bc8c4396c3/src/syrupy/extensions/base.py#L161
        
        filepath, data = (
            snapshot_collection.location,
            next(iter(snapshot_collection)).data,
        )
        np.savetxt(filepath, data)

    def serialize(self, data: SerializableData, **kwargs: Any) -> str:
        return data


class PandasSnapshotExtenstion(SingleFileSnapshotExtension):
    _file_extension = "hdf"

    def matches(self, *, serialized_data, snapshot_data):
        try:
            if pd.testing.assert_frame_equal(
                serialized_data, snapshot_data
            )  is not None:
                return False
            else: return True
            
        except:
            return False

    def _read_snapshot_data_from_location(
        self, *, snapshot_location: str, snapshot_name: str, session_id: str
    ):
        # see https://github.com/tophat/syrupy/blob/f4bc8453466af2cfa75cdda1d50d67bc8c4396c3/src/syrupy/extensions/base.py#L139
        try:
            return pd.read_hdf(snapshot_location)
        except OSError:
            return None

    @classmethod
    def _write_snapshot_collection(
        cls, *, snapshot_collection: SnapshotCollection
    ) -> None:
        # see https://github.com/tophat/syrupy/blob/f4bc8453466af2cfa75cdda1d50d67bc8c4396c3/src/syrupy/extensions/base.py#L161
        filepath, data = (
            snapshot_collection.location,
            next(iter(snapshot_collection)).data,
        )
        data.to_hdf(filepath, "/blah")

    def serialize(self, data: SerializableData, **kwargs: Any) -> str:
        return data


@pytest.fixture
def snapshot_pandas(snapshot):
    return snapshot.use_extension(PandasSnapshotExtenstion)

@pytest.fixture
def snapshot_numpy(snapshot):
    return snapshot.use_extension(NumpySnapshotExtenstion)


def test_pd(snapshot_pandas):
    data = [30,40,60]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

@pytest.mark.parametrize('no', [1,2])
def test_pd3(snapshot_pandas, no):
    data = [30,40,60, no]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

def test_np(snapshot_numpy):
    # assert snapshot_numpy == np.array([1, 3]).tolist()
    # assert snapshot_numpy == np.array([1, 2.5]).tolist()
    assert snapshot_numpy == np.array([1, 3])
    assert snapshot_numpy == np.array([1, 2.5])


======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 4 items

t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd FAILED                                        [ 25%]
t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd3[1] FAILED               

### That was expected. Lets update the snapshots.

In [10]:
%%ipytest -vv --snapshot-update

def test_pd(snapshot_pandas):
    data = [30,40,60]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

@pytest.mark.parametrize('no', [1,2])
def test_pd3(snapshot_pandas, no):
    data = [30,40,60, no]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

def test_np(snapshot_numpy):
    assert snapshot_numpy == np.array([1, 3]).tolist()
    assert snapshot_numpy == np.array([1, 2.5]).tolist()


======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 4 items

t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd PASSED                                        [ 25%]
t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd3[1] PASSED               

### There should be three snapshots.

In [11]:
!tree -L 3

.
├── comp.ipynb
└── __snapshots__
    └── t_6c3a8c59f7a649628d07e268d7a45059
        ├── test_np.1.dat
        ├── test_np.dat
        ├── test_pd3[1].hdf
        ├── test_pd3[2].hdf
        └── test_pd.hdf

2 directories, 6 files


In [13]:
%%ipytest -vv 

def test_pd(snapshot_pandas):
    data = [30,40,60]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

@pytest.mark.parametrize('no', [1,2])
def test_pd3(snapshot_pandas, no):
    data = [30,40,60, no]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

def test_np(snapshot_numpy):
    assert snapshot_numpy == np.array([1, 3]).tolist()
    assert snapshot_numpy == np.array([1, 2.5]).tolist()

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 4 items

t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd PASSED                                        [ 25%]
t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd3[1] PASSED               

### Lets change a few things now.

In [14]:
%%ipytest -vv 

def test_pd(snapshot_pandas):
    data = [30,40,20]
    assert snapshot_pandas == pd.DataFrame(data, columns=['Numbers'])

def test_np(snapshot_numpy):
    assert snapshot_numpy == np.array([1, 2]).tolist()
    assert snapshot_numpy == np.array([1, 2]).tolist()

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 2 items

t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd FAILED                                        [ 50%]
t_6c3a8c59f7a649628d07e268d7a45059.py::test_np FAILED                   

### The methods used won't be changed until a major release.

### Onto pytest regressions.

### Lets look at the directories first. This would help us figure out what created in later steps.

In [15]:
!rm -rf ./t*

zsh:1: no matches found: ./t*


In [16]:
!tree -L 3

.
├── comp.ipynb
└── __snapshots__
    └── t_6c3a8c59f7a649628d07e268d7a45059
        ├── test_np.1.dat
        ├── test_np.dat
        ├── test_pd3[1].hdf
        ├── test_pd3[2].hdf
        └── test_pd.hdf

2 directories, 6 files


### This should fail in the first try but should then succeed.

In [17]:
%%ipytest -vv 
pytest_plugins = "regressions"

# From the docs-

def summary_grids():
    return {
        "Main Grid": {
            "id": 0,
            "cell_count": 1000,
            "active_cells": 300,
            "properties": [
                {"name": "Temperature", "min": 75, "max": 85},
                {"name": "Porosity", "min": 0.3, "max": 0.4},
            ],
        },
        "Refin1": {
            "id": 1,
            "cell_count": 48,
            "active_cells": 44,
            "properties": [
                {"name": "Temperature", "min": 78, "max": 81},
                {"name": "Porosity", "min": 0.36, "max": 0.39},
            ],
        },
    }

def test_grids2(data_regression):
    data = summary_grids()
    data_regression.check(data)

def test_pd(dataframe_regression):
    # you can provide tolerance from arguments.
    # along with path and the name of the file.
    data = [30,40,60]
    dataframe_regression.check(pd.DataFrame(data, columns=['Numbers']))

def test_pd_multindex(dataframe_regression):
    arrays = [
        np.array(["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"]),
        np.array(["one", "two", "one", "two", "one", "two", "one", "two"]),
    ]
    s = pd.Series(np.random.randn(8), index=arrays)
    df = pd.DataFrame(np.random.randn(8, 4), index=arrays)
    
    dataframe_regression.check(df)

def test_file(file_regression):
    stuff = "This needs to be a string."
    file_regression.check(stuff)


def test_num_dict(num_regression):
    num = np.array([1,2,3])
    # num2 = np.array([1,2,3,4]) # The need to be of the same shape, or it yells at you
    # TypeError: Checking multiple arrays with different shapes are not supported for non-float arrays

    num2 = np.array([4,5,6])
    num_dict = {
        "num": num,
        "num2": num2
    }

    # inputs to num_reg have to be dicts.
    num_regression.check(num_dict)


def test_ndarray(ndarrays_regression):
    num = np.array([1,2,3])
    num2 = np.array([4,5,6, 7]) # this can be separates shapes
    num_dict = {
        "num": num,
        "num2": num2
    }
    ndarrays_regression.check(num_dict)
    




======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 6 items

t_6c3a8c59f7a649628d07e268d7a45059.py::test_grids2 FAILED                                    [ 16%]
t_6c3a8c59f7a649628d07e268d7a45059.py::test_pd FAILED                   

### A temp directory should be created-

In [18]:
!tree -L 3

.
├── comp.ipynb
├── __snapshots__
│   └── t_6c3a8c59f7a649628d07e268d7a45059
│       ├── test_np.1.dat
│       ├── test_np.dat
│       ├── test_pd3[1].hdf
│       ├── test_pd3[2].hdf
│       └── test_pd.hdf
└── t_6c3a8c59f7a649628d07e268d7a45059
    ├── test_file.txt
    ├── test_grids2.yml
    ├── test_ndarray.npz
    ├── test_num_dict.csv
    ├── test_pd.csv
    └── test_pd_multindex.csv

3 directories, 12 files


### This was the file created.

In [13]:
cat ./*/test_grids2.yml

Main Grid:
  active_cells: 300
  cell_count: 1000
  id: 0
  properties:
  - max: 85
    min: 75
    name: Temperature
  - max: 0.4
    min: 0.3
    name: Porosity
Refin1:
  active_cells: 44
  cell_count: 48
  id: 1
  properties:
  - max: 81
    min: 78
    name: Temperature
  - max: 0.39
    min: 0.36
    name: Porosity


### Lets change the code and fail this-

In [19]:
%%ipytest -vv 

def summary_grids():
    return {
        "Main Grid": {
            "id": 0,
            "cell_count": 100, # I changed this
            "active_cells": 300,
            "properties": [
                {"name": "Temperature", "min": 75, "max": 85},
                {"name": "Porosity", "min": 0.3, "max": 0.4},
            ],
        },
        "Refin1": {
            "id": 1,
            "cell_count": 48,
            "active_cells": 44,
            "properties": [
                {"name": "Temperature", "min": 78, "max": 81},
                {"name": "Porosity", "min": 0.36, "max": 0.39},
            ],
        },
    }

def test_grids2(data_regression):
    data = summary_grids()
    data_regression.check(data)

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 1 item

t_6c3a8c59f7a649628d07e268d7a45059.py::test_grids2 FAILED                                    [100%]

============================================= FAILURES =================

### But you can also update it when it fails, like so-

In [20]:
%%ipytest -vv  --force-regen

# There is also this command when you want to update all files- when one file is causing others to fail
# %%ipytest -vv   --regen-all


def summary_grids():
    return {
        "Main Grid": {
            "id": 0,
            "cell_count": 100, # I changed this but it didn't fail
            "active_cells": 300,
            "properties": [
                {"name": "Temperature", "min": 75, "max": 85},
                {"name": "Porosity", "min": 0.3, "max": 0.4}
            ],
        },
        "Refin1": {
            "id": 1,
            "cell_count": 48,
            "active_cells": 44,
            "properties": [
                {"name": "Temperature", "min": 78, "max": 81},
                {"name": "Porosity", "min": 0.36, "max": 0.39},
            ],
        },
    }

def test_grids2(data_regression):
    data = summary_grids()
    data_regression.check(data)

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 1 item

t_6c3a8c59f7a649628d07e268d7a45059.py::test_grids2 FAILED                                    [100%]

============================================= FAILURES =================

In [16]:
!cat ./*/test_grids2.yml

Main Grid:
  active_cells: 300
  cell_count: 100
  id: 0
  properties:
  - max: 85
    min: 75
    name: Temperature
  - max: 0.4
    min: 0.3
    name: Porosity
Refin1:
  active_cells: 44
  cell_count: 48
  id: 1
  properties:
  - max: 81
    min: 78
    name: Temperature
  - max: 0.39
    min: 0.36
    name: Porosity


### Multiple files can be created if you want, using pytest parametrization.

In [17]:
%%ipytest -vv  

def summary_grids_2():
    return {
        "Main Grid": {
            "id": 0,
            "cell_count": 200, # I changed this
            "active_cells": 300,
            "properties": [
                {"name": "Temperature", "min": 75, "max": 85},
                {"name": "Porosity", "min": 0.3, "max": 0.4},
            ],
        },
        "Refin1": {
            "id": 1,
            "cell_count": 48,
            "active_cells": 44,
            "properties": [
                {"name": "Temperature", "min": 78, "max": 81},
                {"name": "Porosity", "min": 0.36, "max": 0.39},
            ],
        },
    }



@pytest.mark.parametrize('data', [summary_grids(), summary_grids_2()])
def test_grids3(data_regression, data):
    data_regression.check(data)

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 2 items

t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data0] FAILED                             [ 50%]
t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data1] FAILED        

In [18]:
%%ipytest -vv  
@pytest.mark.parametrize('data', [summary_grids(), summary_grids_2()])
def test_grids3(data_regression, data):
    data_regression.check(data)

======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 2 items

t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data0] PASSED                             [ 50%]
t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data1] PASSED        

In [73]:
!tree -L 3

.
├── comp.ipynb
├── __snapshots__
│   └── t_16e71ae9488a43a698f211ebd4a48f7b
│       ├── test_np.1.dat
│       ├── test_np.dat
│       └── test_pd.hdf
└── t_16e71ae9488a43a698f211ebd4a48f7b
    ├── test_file.txt
    ├── test_grids2.yml
    ├── test_ndarray.npz
    ├── test_num_dict.csv
    ├── test_pd.csv
    └── test_pd_multindex.csv

3 directories, 10 files


In [74]:
cat ./*/test_pd_multindex.csv

,,0,1,2,3
bar,one,-1.1014007501847189,1.2137688421392367,0.89851417512235565,0.17964640157624542
bar,two,0.32081546833621222,0.66701932492200178,-0.60722680298514653,-0.03174233281382681
baz,one,0.34341894268733292,-0.96087434776995451,-0.26824298275965186,-1.156710348595575
baz,two,-0.89954644564232367,-0.65340736286093082,0.066026078838057126,-0.73536157201033336
foo,one,-0.37712096837915737,1.0536687258531683,0.41687973150755075,1.8542702380273814
foo,two,0.79467857874741532,-0.12401699644480658,1.4692128281040648,0.22272087726281331
qux,one,0.11959572469826744,0.1779897914459474,-0.38707770401268371,1.1616930612202028
qux,two,1.5006516300855264,1.4187690016693142,-2.6288908767549182,0.25280984421207281


In [76]:
pd.read_csv("./t_16e71ae9488a43a698f211ebd4a48f7b/test_pd_multindex.csv")

,Unnamed: 0,Unnamed: 1,0,1,2,3
0,bar,one,-1.101401,1.213769,0.898514,0.179646
1,bar,two,0.320815,0.667019,-0.607227,-0.031742
2,baz,one,0.343419,-0.960874,-0.268243,-1.156710
3,baz,two,-0.899546,-0.653407,0.066026,-0.735362
4,foo,one,-0.377121,1.053669,0.416880,1.854270
5,foo,two,0.794679,-0.124017,1.469213,0.222721
6,qux,one,0.119596,0.177990,-0.387078,1.161693
7,qux,two,1.500652,1.418769,-2.628891,0.252810


In [21]:
%%ipytest -vv  
@pytest.mark.parametrize('data', [summary_grids(), summary_grids_2()])
def test_grids3(data_regression, data):
    data_regression.check(data)


======================================= test session starts ========================================
platform linux -- Python 3.8.17, pytest-7.4.0, pluggy-1.2.0 -- /home/atharva/miniconda3/envs/tardis-devel/bin/python
cachedir: .pytest_cache
metadata: {'Python': '3.8.17', 'Platform': 'Linux-5.15.0-57-generic-x86_64-with-glibc2.10', 'Packages': {'pytest': '7.4.0', 'pluggy': '1.2.0'}, 'Plugins': {'datadir': '1.4.1', 'anyio': '3.7.1', 'cov': '4.1.0', 'arraydiff': '0.6.0a1', 'metadata': '3.0.0', 'html': '3.2.0', 'syrupy': '4.0.8', 'regressions': '2.4.2', 'doctestplus': '1.0.0'}}
rootdir: /home/atharva/workspace/code/tardis-main/syrupy-reg-comp
plugins: datadir-1.4.1, anyio-3.7.1, cov-4.1.0, arraydiff-0.6.0a1, metadata-3.0.0, html-3.2.0, syrupy-4.0.8, regressions-2.4.2, doctestplus-1.0.0
collecting ... collected 2 items

t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data0] PASSED                             [ 50%]
t_2019e8df2c304aa4818ff11afd15cceb.py::test_grids3[data1] PASSED        